In [2]:
print("hi")

hi


In [3]:
import os
os.environ['OPENAI_API_KEY'] = 'REDACTED'

In [4]:
# from langchain.chains.sql_database.prompt import SQL_PROMPTS

# list(SQL_PROMPTS)

In [5]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_community.utilities import SQLDatabase

In [7]:
# db_uri_pg_chinook = 'postgresql://hand@localhost:5432/chinook'
# db_pg_chinook = SQLDatabase.from_uri(db_uri_pg_chinook)

In [8]:
# db_pg_chinook.run("SELECT * from album LIMIT 5")

In [9]:
# db_pg_chinook.run('select version()')

In [10]:
pip install sqlalchemy-cockroachdb

Note: you may need to restart the kernel to use updated packages.


In [11]:
# db_uri_crl_chinook = 'cockroachdb://root@crlMBP-NRWLD7WPC6MTc1.local:26257/chinook?sslmode=disable&options=-c%20search_path=crdb_internal,public'
# db_crl_chinook = SQLDatabase.from_uri(db_uri_crl_chinook)

In [12]:
db_uri_crl_chinook = 'cockroachdb://root@crlMBP-NRWLD7WPC6MTc1.local:26257/chinook?sslmode=disable'
db_crl_chinook = SQLDatabase.from_uri(db_uri_crl_chinook)

In [13]:
db_crl_chinook.run("SELECT 10")

'[(10,)]'

In [15]:
db_crl_chinook.run("show databases")

"[('chinook', 'root', None, None, [], None), ('defaultdb', 'root', None, None, [], None), ('postgres', 'root', None, None, [], None), ('system', 'node', None, None, [], None)]"

In [21]:
db_crl_chinook.run("show tables")

"[('public', 'album', 'table', 'root', 0, None), ('public', 'artist', 'table', 'root', 275, None), ('public', 'customer', 'table', 'root', 0, None), ('public', 'employee', 'table', 'root', 0, None), ('public', 'genre', 'table', 'root', 0, None), ('public', 'invoice', 'table', 'root', 412, None), ('public', 'invoice_line', 'table', 'root', 2240, None), ('public', 'media_type', 'table', 'root', 0, None), ('public', 'playlist', 'table', 'root', 0, None), ('public', 'playlist_track', 'table', 'root', 8715, None), ('public', 'track', 'table', 'root', 3503, None)]"

In [22]:
db_crl_chinook.run("select version()")

"[('CockroachDB CCL v24.1.4 (aarch64-apple-darwin21.2, built 2024/08/26 19:15:47, go1.22.5 X:nocoverageredesign)',)]"

In [23]:
db_crl_chinook.run("select count(*) from album")

'[(347,)]'

In [24]:
db_crl_chinook.run("select * from album limit 5")

"[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3)]"

In [25]:
db_crl_chinook.run("select count(*) from crdb_internal.cluster_contention_events;")

'[(5,)]'

In [26]:
db_crl_chinook.run("SELECT SUM(num_contention_events) AS total_contention_events FROM crdb_internal.cluster_contention_events;")

"[(Decimal('13'),)]"

In [27]:
db_crl_chinook.get_table_info()

'\nCREATE TABLE album (\n\talbum_id INTEGER NOT NULL, \n\ttitle VARCHAR(160) NOT NULL, \n\tartist_id INTEGER NOT NULL, \n\tCONSTRAINT album_pkey PRIMARY KEY (album_id), \n\tCONSTRAINT album_artist_id_fkey FOREIGN KEY(artist_id) REFERENCES artist (artist_id)\n)\n\n/*\n3 rows from album table:\nalbum_id\ttitle\tartist_id\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE artist (\n\tartist_id INTEGER NOT NULL, \n\tname VARCHAR(120), \n\tCONSTRAINT artist_pkey PRIMARY KEY (artist_id)\n)\n\n/*\n3 rows from artist table:\nartist_id\tname\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE customer (\n\tcustomer_id INTEGER NOT NULL, \n\tfirst_name VARCHAR(40) NOT NULL, \n\tlast_name VARCHAR(20) NOT NULL, \n\tcompany VARCHAR(80), \n\taddress VARCHAR(70), \n\tcity VARCHAR(40), \n\tstate VARCHAR(40), \n\tcountry VARCHAR(40), \n\tpostal_code VARCHAR(10), \n\tphone VARCHAR(24), \n\tfax VARCHAR(24), \n\temail VARCHAR(60) NOT NU

In [28]:
db_crl_chinook.get_usable_table_names()

['album',
 'artist',
 'customer',
 'employee',
 'genre',
 'invoice',
 'invoice_line',
 'media_type',
 'playlist',
 'playlist_track',
 'track']

In [30]:
from langchain_core.prompts import ChatPromptTemplate

In [31]:
template_question_to_sql = """
Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:
"""

prompt_question_to_sql = ChatPromptTemplate.from_template(template_question_to_sql)

In [32]:
prompt_question_to_sql.format(schema="my schema", question="how many users are there?")

"Human: \nBased on the table schema below, write a SQL query that would answer the user's question:\nmy schema\n\nQuestion: how many users are there?\nSQL Query:\n"

In [33]:
def get_schema_crl_chinook(_):
    return db_crl_chinook.get_table_info()

In [34]:
get_schema_crl_chinook(None)

'\nCREATE TABLE album (\n\talbum_id INTEGER NOT NULL, \n\ttitle VARCHAR(160) NOT NULL, \n\tartist_id INTEGER NOT NULL, \n\tCONSTRAINT album_pkey PRIMARY KEY (album_id), \n\tCONSTRAINT album_artist_id_fkey FOREIGN KEY(artist_id) REFERENCES artist (artist_id)\n)\n\n/*\n3 rows from album table:\nalbum_id\ttitle\tartist_id\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE artist (\n\tartist_id INTEGER NOT NULL, \n\tname VARCHAR(120), \n\tCONSTRAINT artist_pkey PRIMARY KEY (artist_id)\n)\n\n/*\n3 rows from artist table:\nartist_id\tname\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE customer (\n\tcustomer_id INTEGER NOT NULL, \n\tfirst_name VARCHAR(40) NOT NULL, \n\tlast_name VARCHAR(20) NOT NULL, \n\tcompany VARCHAR(80), \n\taddress VARCHAR(70), \n\tcity VARCHAR(40), \n\tstate VARCHAR(40), \n\tcountry VARCHAR(40), \n\tpostal_code VARCHAR(10), \n\tphone VARCHAR(24), \n\tfax VARCHAR(24), \n\temail VARCHAR(60) NOT NU

In [35]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [36]:
llm = ChatOpenAI(temperature=0)

In [37]:
sql_chain_crl_chinook = (
    RunnablePassthrough.assign(schema=get_schema_crl_chinook)
    | prompt_question_to_sql
    | llm.bind(stop="\nSQL Result:")
    | StrOutputParser()
)

In [38]:
sql_chain_crl_chinook.invoke({"question": "how many artists are there?"})

'SELECT COUNT(artist_id) AS total_artists FROM artist;'

In [40]:
template_question_to_nl = """
Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""

prompt_question_to_nl = ChatPromptTemplate.from_template(template_question_to_nl)

In [41]:
def run_query_crl_chinook(query):
    return db_crl_chinook.run(query)

In [42]:
run_query_crl_chinook('SELECT COUNT(*) AS total_artists FROM Artist;')

'[(275,)]'

In [43]:
full_chain_crl_chinook_1 = (
    RunnablePassthrough.assign(query=sql_chain_crl_chinook).assign(
        schema=get_schema_crl_chinook,
        response=lambda variables: print(variables)
    )
)

In [44]:
full_chain_crl_chinook_1.invoke({"question": "how many artists are there?"})

{'question': 'how many artists are there?', 'query': 'SELECT COUNT(artist_id) AS total_artists FROM artist;'}


{'question': 'how many artists are there?',
 'query': 'SELECT COUNT(artist_id) AS total_artists FROM artist;',
 'schema': '\nCREATE TABLE album (\n\talbum_id INTEGER NOT NULL, \n\ttitle VARCHAR(160) NOT NULL, \n\tartist_id INTEGER NOT NULL, \n\tCONSTRAINT album_pkey PRIMARY KEY (album_id), \n\tCONSTRAINT album_artist_id_fkey FOREIGN KEY(artist_id) REFERENCES artist (artist_id)\n)\n\n/*\n3 rows from album table:\nalbum_id\ttitle\tartist_id\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE artist (\n\tartist_id INTEGER NOT NULL, \n\tname VARCHAR(120), \n\tCONSTRAINT artist_pkey PRIMARY KEY (artist_id)\n)\n\n/*\n3 rows from artist table:\nartist_id\tname\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE customer (\n\tcustomer_id INTEGER NOT NULL, \n\tfirst_name VARCHAR(40) NOT NULL, \n\tlast_name VARCHAR(20) NOT NULL, \n\tcompany VARCHAR(80), \n\taddress VARCHAR(70), \n\tcity VARCHAR(40), \n\tstate VARCHAR(40), \n\t

In [45]:
full_chain_crl_chinook_2 = (
    RunnablePassthrough.assign(query=sql_chain_crl_chinook).assign(
        schema=get_schema_crl_chinook,
        response=lambda variables: run_query_crl_chinook(variables["query"])
    )
    | prompt_question_to_nl
    | llm
)

In [46]:
full_chain_crl_chinook_2.invoke({"question": "how many artists are there?"})

AIMessage(content='There are a total of 275 artists in the database.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 2102, 'total_tokens': 2115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-95f0ab07-6ddc-4076-b2b5-da5dd11b3ee4-0', usage_metadata={'input_tokens': 2102, 'output_tokens': 13, 'total_tokens': 2115, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [47]:
full_chain_crl_chinook_3 = (
    RunnablePassthrough.assign(query=sql_chain_crl_chinook).assign(
        schema=get_schema_crl_chinook,
        response=lambda vars: run_query_crl_chinook(vars["query"])
    )
    | prompt_question_to_nl
    | llm
    | StrOutputParser()
)

In [48]:
full_chain_crl_chinook_3.invoke({"question": "how many artists are there?"})

'There are a total of 275 artists in the database.'

In [52]:
print('----------------------------------------------------------------------------------------------------------')

----------------------------------------------------------------------------------------------------------


In [49]:
with open("/Users/hand/Documents/projects/ChatGPT/contention/contention_table_schema_and_guidance.txt", "r") as file:
    schema_and_guidance = file.read()

In [50]:
print(schema_and_guidance[:200])

You are a relational database expert.  I will tell you a database schema and explain what the tables mean.  Then you will give me SQL queries for the data I seek.

I am using the CockroachDB database.


In [51]:
def get_schema_contention(_):
    return schema_and_guidance

In [52]:
print(get_schema_contention(None)[:200])

You are a relational database expert.  I will tell you a database schema and explain what the tables mean.  Then you will give me SQL queries for the data I seek.

I am using the CockroachDB database.


In [53]:
prompt_question_to_sql.format(schema="my schema", question="how many contention events are there?")

"Human: \nBased on the table schema below, write a SQL query that would answer the user's question:\nmy schema\n\nQuestion: how many contention events are there?\nSQL Query:\n"

In [54]:
# llm = ChatOpenAI()

In [55]:
sql_chain_contention = (
    RunnablePassthrough.assign(schema=get_schema_contention)
    | prompt_question_to_sql
    | llm.bind(stop="\nSQL Result:")
    | StrOutputParser()
)

In [56]:
sql_chain_contention.invoke({"question": "how many contention events are there?"})

'SELECT SUM(num_contention_events) AS total_contention_events\nFROM crdb_internal.cluster_contention_events;'

In [57]:
full_chain_contention_1 = (
    RunnablePassthrough.assign(query=sql_chain_contention).assign(
        schema=get_schema_contention,
        response=lambda variables: print(variables)
    )
)

In [58]:
full_chain_contention_1.invoke({"question": "how many contention events are there?"})

{'question': 'how many contention events are there?', 'query': 'SELECT SUM(num_contention_events) AS total_contention_events\nFROM crdb_internal.cluster_contention_events;'}


{'question': 'how many contention events are there?',
 'query': 'SELECT SUM(num_contention_events) AS total_contention_events\nFROM crdb_internal.cluster_contention_events;',
 'schema': 'You are a relational database expert.  I will tell you a database schema and explain what the tables mean.  Then you will give me SQL queries for the data I seek.\n\nI am using the CockroachDB database.  The SQL I will use will be compatible with CockroachDB.\n\nHere is the first table definition:\n\nCREATE TABLE crdb_internal.cluster_contention_events (\n      table_id INT8 NULL,\n      index_id INT8 NULL,\n      num_contention_events INT8 NOT NULL,\n      cumulative_contention_time INTERVAL NOT NULL,\n      key BYTES NOT NULL,\n      txn_id UUID NOT NULL,\n      count INT8 NOT NULL\n  )\n\nHere is another table:\n\nCREATE TABLE crdb_internal.transaction_contention_events (\n      collection_ts TIMESTAMPTZ NOT NULL,\n      blocking_txn_id UUID NOT NULL,\n      blocking_txn_fingerprint_id BYTES NOT NUL

In [59]:
full_chain_contention_2 = (
    RunnablePassthrough.assign(query=sql_chain_contention).assign(
        schema=get_schema_contention,
        response=lambda variables: run_query_crl_chinook(variables["query"])
    )
    | prompt_question_to_nl
    | llm
)

In [60]:
full_chain_contention_2.invoke({"question": "how many contention events are there?"})

AIMessage(content='There are a total of 13 contention events in the cluster_contention_events table.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 2551, 'total_tokens': 2569, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6841a935-2b8a-4105-92ae-c2314d2d5ec5-0', usage_metadata={'input_tokens': 2551, 'output_tokens': 18, 'total_tokens': 2569, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [61]:
full_chain_contention_3 = (
    RunnablePassthrough.assign(query=sql_chain_contention).assign(
        schema=get_schema_contention,
        response=lambda vars: run_query_crl_chinook(vars["query"])
    )
    | prompt_question_to_nl
    | llm
    | StrOutputParser()
)

In [62]:
full_chain_contention_3.invoke({"question": "how many contention events are there?"})

'There are a total of 13 contention events in the cluster_contention_events table.'